<a href="https://colab.research.google.com/github/chitkalaakella/Coding-with-AI/blob/main/Planetary_Ray_Angle_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import importlib
import sys

# Check if astropy is installed
if importlib.util.find_spec('astropy') is None:
    print("Error: The Astropy library is required to run this code.")
    print("Please install it using pip:")
    print("  " + sys.executable + " -m pip install astropy")
    sys.exit()

from astropy.coordinates import get_body, EarthLocation, SkyCoord
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import solar_system_ephemeris, AffineTransform
from scipy.spatial.transform import Rotation as R

def get_planet_earth_heeq(planet_name, time):
    """
    Gets the Heliocentric Earth Equatorial (HEEQ) coordinates of a planet and Earth.

    Args:
        planet_name (str): Name of the planet.
        time (Time): Astropy Time object for the desired time.

    Returns:
        tuple: (planet_heeq, earth_heeq) as astropy CartesianRepresentation objects,
               or (None, None) if a planet name is invalid.
    """
    if planet_name.lower() == 'earth':
        planet_loc = EarthLocation.of_site('greenwich')
        planet_pos = get_body('earth', time, location=planet_loc)
        earth_pos = planet_pos  # Earth's HEEQ is relative to the Sun
    else:
        planet_loc = EarthLocation.of_site('greenwich')
        planet_pos = get_body(planet_name, time, location=planet_loc)
        earth_pos = get_body('earth', time, location=planet_loc)
        sun_pos = get_body('sun', time, location=planet_loc)

    # 1. Get heliocentric ecliptic coordinates.
    planet_ecliptic = planet_pos.heliocentrictrueecliptic
    #planet_ecliptic = planet_ecliptic.transform_to('icrs')
    earth_ecliptic = earth_pos.heliocentrictrueecliptic
    sun_ecliptic = sun_pos.heliocentrictrueecliptic

    # 2. Define the transformation to HEEQ.  This requires several rotations.
    #    See https://www.mssl.ucl.ac.uk/grid/iau/extra/local_copy/SP_coords/helitran.htm
    #    and https://en.wikipedia.org/wiki/Heliocentric_Earth_equatorial_coordinate_system

    # a. Rotation about Z-axis to align with the Sun's ecliptic longitude.
    sun_lon = sun_ecliptic.lon
    rot_z_lon = R.from_euler('z', sun_lon.deg, degrees=True)

    # b. Rotation about X-axis by the obliquity of the ecliptic (tilt of Earth's axis).
    obliquity = 23.4397 * u.deg  # Approximate value, can be time-dependent for better accuracy
    rot_x_obliquity = R.from_euler('x', obliquity, degrees=True)

    # c. Rotation about Z-axis by the longitude of the ascending node of the solar equator
    #    on the ecliptic.  This and the solar equator inclination define the orientation
    #    of the Sun's equator.
    solar_equator_asc_node = 75.7 * u.deg  # Approximate value.
    rot_z_node = R.from_euler('z', solar_equator_asc_node.deg, degrees=True)

    # d. Rotation about X-axis by the inclination of the solar equator to the ecliptic.
    solar_equator_inclination = 7.25 * u.deg  # Approximate value
    rot_x_inclination = R.from_euler('x', obliquity.deg, degrees=True)
    # Combine the rotations.  The order is important.
    # The order of rotations is: 1. align with sun's longitude, 2. tilt to ecliptic,
    # 3. rotate to solar node, 4. tilt to solar equator.
    heeq_matrix = rot_x_inclination.as_matrix() @ rot_z_node.as_matrix() @ rot_x_obliquity.as_matrix() @ rot_z_lon.as_matrix()
    to_heeq = AffineTransform(matrix=heeq_matrix)

    planet_heeq = planet_ecliptic.transform_to(to_heeq)
    earth_heeq = earth_ecliptic.transform_to(to_heeq)

    return planet_heeq.cartesian.xyz.to(u.AU).value, earth_heeq.cartesian.xyz.to(u.AU).value


def calculate_intersection_angle(planet_pos_heeq, earth_pos_heeq, earth_intersect_point_heeq,
                               planet_face_center, time):
    """
    Calculates the angle between the ray from the center of a planet's face to a
    point on Earth's equator and the Earth's equatorial plane at that point.
    This is done in the HEEQ frame.

    Args:
        planet_pos_heeq (np.ndarray): Planet's HEEQ coordinates (x, y, z) in AU.
        earth_pos_heeq (np.ndarray): Earth's HEEQ coordinates (x, y, z) in AU.
        earth_intersect_point_heeq (np.ndarray): Point on Earth's surface in HEEQ (x,y,z) in AU.
        planet_face_center (np.ndarray):  The center point of the planet's face
                                          relative to the planet's center, in the same
                                          HEEQ coordinates as planet_pos_heeq.
        time (Time): The time at which the calculation is done.

    Returns:
        float: Angle in degrees, or None if vectors are zero.
    """

    # Calculate the ray vector from the center of the planet's face.
    ray_vector = earth_intersect_point_heeq - (planet_pos_heeq + planet_face_center)
    earth_center_heeq = earth_pos_heeq

    # 1.  Find the normal vector to the Earth's surface at the intersection point.
    surface_normal = (earth_intersect_point_heeq - earth_center_heeq)
    if np.linalg.norm(surface_normal) == 0:
        return None
    unit_normal = surface_normal / np.linalg.norm(surface_normal)

    # 2. Define Earth's equatorial plane in HEEQ.
    #    In HEEQ, the Z-axis is aligned with the Sun's rotation axis.
    #    We need to find a vector that is perpendicular to Earth's equatorial plane
    #    *at the location of the intersection point*.
    #    This is NOT necessarily the same as the HEEQ Z-axis.
    #
    #    To do this correctly, we need to use the Earth's rotation at the given time.
    #    We can get this from Astropy.  We'll use the ITRS frame, and then transform
    #    a vector from that frame to HEEQ.

    earth_location = EarthLocation.from_vector(earth_intersect_point_heeq * u.AU)  # Create EarthLocation

    # Get the unit vector representing the Z-axis in the ITRS frame
    itrs_z_axis = np.array([0, 0, 1])

    # Create a SkyCoord object representing the Z-axis in ITRS
    itrs_z_coord = SkyCoord(x=itrs_z_axis[0], y=itrs_z_axis[1], z=itrs_z_axis[2], frame='itrs', obstime=time)
    # Transform this vector to HEEQ
    icrs_z_coord = itrs_z_coord.transform_to('icrs')
    heeq_z_coord = icrs_z_coord.cartesian.xyz.value

    # Normalize the vector.
    unit_equatorial_normal = heeq_z_coord / np.linalg.norm(heeq_z_coord)

    # 3. Calculate the angle.
    dot_product_surface = np.dot(ray_vector, unit_normal)
    angle_with_surface_rad = np.arccos(dot_product_surface / (np.linalg.norm(ray_vector) * np.linalg.norm(unit_normal)))
    angle_with_surface_deg = np.degrees(angle_with_surface_rad)
    angle_from_surface = 90.0 - angle_with_surface_deg

    dot_product_equator = np.dot(ray_vector, unit_equatorial_normal)
    angle_with_equatorial_rad = np.arccos(
        dot_product_equator / (np.linalg.norm(ray_vector) * np.linalg.norm(unit_equatorial_normal)))
    angle_with_equatorial_deg = np.degrees(angle_with_equatorial_rad)

    return angle_with_equatorial_deg  # , angle_from_surface # Return both angles



def find_45_degree_angle(planet_name, time, max_iterations=1000, tolerance=0.1):
    """
    Finds a point on Earth's equator where the angle of the ray from the planet's
    face center to that point is approximately 45 degrees to Earth's equatorial
    plane.  It returns the angle and the number of iterations.

    Args:
        planet_name (str): Name of the planet.
        time (Time): Astropy Time object.
        max_iterations (int): Maximum number of iterations to search.
        tolerance (float):  Maximum allowed difference from 45 degrees.

    Returns:
        tuple: (angle, num_iterations), or (None, None) if not found.
    """
    planet_heeq, earth_heeq = get_planet_earth_heeq(planet_name, time)
    if planet_heeq is None or earth_heeq is None:
        return None, None

    earth_radius_au = 6371 / 149597870.7  # Earth radius in AU
    planet_radius_au = 3389.5 / 149597870.7 if planet_name.lower() == "mars" else 6051.8 / 149597870.7
    planet_face_center = np.array([planet_radius_au, 0, 0])

    # Search along Earth's equator.  We'll start at longitude 0 and iterate.
    num_iterations = 0
    for iteration in range(max_iterations):
        num_iterations += 1
        lon = iteration * (360.0 / max_iterations)  # Spread longitudes evenly
        itrs_equator_point = EarthLocation.from_geodetic(lon=lon * u.deg, lat=0 * u.deg, height=0 * u.m,
                                                        obstime=time)
        equator_point_icrs = SkyCoord(itrs_equator_point.get_vector(time=time), frame='itrs')
        equator_point_heeq_coord = equator_point_icrs.transform_to('icrs')
        equator_point_heeq = earth_heeq + equator_point_heeq_coord.cartesian.xyz.to(u.AU).value

        angle = calculate_intersection_angle(planet_heeq, earth_heeq, equator_point_heeq,
                                            planet_face_center, time)
        if angle is not None:
            if np.abs(angle - 45) <= tolerance:
                return angle, num_iterations

    return None, num_iterations  # Return None if not found within max_iterations



# --- Main Execution ---
if __name__ == "__main__":
    planet_names = ["mars", "venus", "jupiter"]  # Add more planets as needed
    time = Time("2025-04-22T12:00:00", format='isot', scale='utc')

    for planet_name in planet_names:
        angle, num_iterations = find_45_degree_angle(planet_name, time)
        if angle is not None:
            print(f"For {planet_name}, found angle {angle:.2f} degrees after {num_iterations} iterations.")
        else:
            print(f"For {planet_name}, could not find 45-degree angle within maximum iterations ({num_iterations}).")

AttributeError: 'Quantity' object has no 'deg' member